## Tutorial - Extracting Text and Analysing Sentiment from Digitized Documents in the UCSD Library's Digital Archive

### Part 4 - Analyzing Categories

In this section of the tutorial, we'll look into the relationship between the documents we've extracted, the accuracy and completion rates, and the categories assigned to each document by the GCP natural language processing API's pre-trained classification model. 

### Normalizing the dataset

First thing you might notice is that this is a difficult dataset to query. Let's read the classification data into a dataframe.

In [0]:
import pandas as pd
from google.colab import drive

as before, we need to mount the drive to get access to the csv file created and saved in part 2

In [2]:
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
df_categories = pd.read_csv('gdrive/My Drive//No-More-Silence/glbths_2005-13_001_001_categories.csv')

Take a look at the Categories column. The data returned from the API is in JSON format, which corresponds to a dictionary in Python.

Unfortunately, this will make it difficult to query records based on a particular category (in relational database design, this means the table is not in first normal form - each cell does not represent a "single, atomic (indivisible) value"[1]).

[1] https://en.wikipedia.org/wiki/First_normal_form

In [4]:
df_categories

,Unnamed: 0,document_page_id,Extracted,Cleaned,Categories
0,0,glbths_2005-13_001_001.pdf_0,"July 24, 1990\nDe ar Sue,\nIhe Louisiana State...",july 24 de ar sue louisiana state penitentiary...,{'categories': [{'name': '/Health/Health Condi...
1,1,glbths_2005-13_001_001.pdf_1,Since then there have been absolutely no effor...,since then there have been absolutely no effor...,{'categories': []}
2,2,glbths_2005-13_001_001.pdf_2,And be cause of his inability to intellectuall...,and be cause of his inability to intellectuall...,{'categories': []}
3,3,glbths_2005-13_001_001.pdf_3,"Sue,\nIf possible will you consider sending me...",sue if possible will you consider sending me a...,"{'categories': [{'name': '/People & Society', ..."
4,4,glbths_2005-13_001_001.pdf_4,TOUGE\nPM\nAlbert Chui Clark\n79909\nČK D_Tier...,pm albert clark cell 4 louisiana state peniten...,{'categories': []}
5,5,glbths_2005-13_001_001.pdf_5,NOT CENSORED\nNot Responsible for Contents\nLa...,not censored not responsible for contents la p...,NaN
6,6,glbths_2005-13_001_001.pdf_6,"Jan '91\nDean An,\nklay\nLow was your\nchristm...",jan 91 dean an low was your christmas d t seen...,{'categories': [{'name': '/People & Society/Re...
7,7,glbths_2005-13_001_001.pdf_7,tr as d lanen t develaped any of\nany\nThe com...,as d t any of any the to 90 l n till year an g...,{'categories': []}
8,8,glbths_2005-13_001_001.pdf_8,C NESON\nAUSTIN\nPM\nイX 78\nRiCHARD\nus# 78826...,c austin pm 78 us lint p 0 box tx usa 2 jan se...,NaN
9,9,glbths_2005-13_001_001.pdf_9,Doud Fletcher\nWorksatRles\nta Publec Health O...,fletcher ta health talk that each its i 7 some...,"{'categories': [{'name': '/Health', 'confidenc..."


### Create a table with one category per line. 

Database design is an intetesting topic. For this tutorial, we'll create a separate row for each unique document-page-id, category, and probability (in RDBMS terms, we'll get this into first normal form). 

In [0]:
import json

In [0]:
document_page_ids = []
category_names = []
category_confidences = []

for d in df_categories.itertuples():
  if type(d[5]) == type('str'):
    json_acceptable_string = d[5].replace("'", "\"")
    jsn = json.loads(json_acceptable_string)
    for c in jsn['categories']:
      document_page_ids.append(d[2])
      category_names.append(c['name'])
      category_confidences.append(c['confidence'])

In [0]:
df_categories_1f = pd.DataFrame({"document_page_id": document_page_ids, "category": category_names, "confidence": category_confidences})

In [8]:
df_categories_1f

,document_page_id,category,confidence
0,glbths_2005-13_001_001.pdf_0,/Health/Health Conditions/Infectious Diseases,0.93
1,glbths_2005-13_001_001.pdf_0,/Health/Reproductive Health,0.93
2,glbths_2005-13_001_001.pdf_0,/Health/Health Conditions/AIDS & HIV,0.81
3,glbths_2005-13_001_001.pdf_3,/People & Society,0.65
4,glbths_2005-13_001_001.pdf_6,/People & Society/Religion & Belief,0.57
5,glbths_2005-13_001_001.pdf_9,/Health,0.67
6,glbths_2005-13_001_001.pdf_11,/Health/Health Conditions/Infectious Diseases,0.89
7,glbths_2005-13_001_001.pdf_11,/Health/Health Conditions/AIDS & HIV,0.87
8,glbths_2005-13_001_001.pdf_11,/Health/Reproductive Health,0.87
9,glbths_2005-13_001_001.pdf_12,/Health/Health Conditions/Infectious Diseases,0.88


### Investigating the relationshop between records and categories, retention rate, and confidence

Outer join to get all rows

In [0]:
df = pd.read_csv('gdrive/My Drive//No-More-Silence/glbths_2005-13_001_001.csv')

In [12]:
print(df_categories_1f.columns)
print(df.columns)

Index(['document_page_id', 'category', 'confidence'], dtype='object')
Index(['Unnamed: 0', 'Sentiment', 'Magnitude', 'Cleaned', 'Retained',
       'Extracted', 'document_page_id'],
      dtype='object')


In [0]:
#df_categories['document_page_id']

In [16]:
!pip install pandasql

  Created wheel for pandasql: filename=pandasql-0.7.3-cp36-none-any.whl size=26819 sha256=b5acef69a6f3f3ad2f2e08c98183f36113d852cf5ea1e503f427945df80d1141
  Stored in directory: /root/.cache/pip/wheels/53/6c/18/b87a2e5fa8a82e9c026311de56210b8d1c01846e18a9607fc9
Successfully built pandasql


In [0]:
from pandasql import sqldf
pysqldf = lambda q: sqldf(q, globals())

In [0]:
df_join = pysqldf("SELECT d.document_page_id, d.retained, dc.document_page_id, dc.category, dc.confidence FROM \
  df d LEFT OUTER JOIN df_categories_1f dc ON d.document_page_id = dc.document_page_id")

In [19]:
df_join

,document_page_id,Retained,document_page_id,category,confidence
0,glbths_2005-13_001_001.pdf_0,0.775498,glbths_2005-13_001_001.pdf_0,/Health/Health Conditions/AIDS & HIV,0.81
1,glbths_2005-13_001_001.pdf_0,0.775498,glbths_2005-13_001_001.pdf_0,/Health/Health Conditions/Infectious Diseases,0.93
2,glbths_2005-13_001_001.pdf_0,0.775498,glbths_2005-13_001_001.pdf_0,/Health/Reproductive Health,0.93
3,glbths_2005-13_001_001.pdf_1,0.796132,None,None,NaN
4,glbths_2005-13_001_001.pdf_2,0.806931,None,None,NaN
5,glbths_2005-13_001_001.pdf_3,0.842022,glbths_2005-13_001_001.pdf_3,/People & Society,0.65
6,glbths_2005-13_001_001.pdf_4,0.562500,None,None,NaN
7,glbths_2005-13_001_001.pdf_5,0.848739,None,None,NaN
8,glbths_2005-13_001_001.pdf_6,0.504621,glbths_2005-13_001_001.pdf_6,/People & Society/Religion & Belief,0.57
9,glbths_2005-13_001_001.pdf_7,0.542120,None,None,NaN


In [20]:
pysqldf("SELECT category, COUNT(*), AVG(retained), AVG(confidence) FROM df_join GROUP BY category ORDER BY COUNT(*) DESC")

,category,COUNT(*),AVG(retained),AVG(confidence)
0,None,20,0.599253,NaN
1,/Health/Reproductive Health,6,0.765560,0.806667
2,/Health/Health Conditions/Infectious Diseases,4,0.823089,0.852500
3,/Health/Health Conditions/AIDS & HIV,3,0.811017,0.853333
4,/Health,2,0.600653,0.595000
5,/People & Society,2,0.825919,0.620000
6,/Sensitive Subjects,2,0.677070,0.610000
7,/Arts & Entertainment,1,0.588710,0.540000
8,/Arts & Entertainment/Music & Audio,1,0.632716,0.580000
9,/Beauty & Fitness,1,0.632622,0.610000
